# Example 3: Computation Utilizing IBM Qiskit Runtime

If you're interested, how to run SA-OO-VQE with real quantum computers, it's about using
`Estimator` and `Sampler` from [IBM Qiskit Runtime](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime)
platform. The following tutorial will show you, how to set your problem up.

WARNING: As all the measurements are being performed on IBM servers, the problem may take really
long time to finish. The more practical way would be to run the code on some hybrid quantum-classical
cluster or utilizing IBM Cloud.

First of all, we'll specify geometry of the system.

In [ ]:
geometry = [('H', [0, 0, 0]),
            ('H', [0, 0, 0.735])]

Now we'll specify its properties, active space and the basis for Psi4 chemistry backend.

In [ ]:
n_orbs_active = 2
n_elec_active = 2
charge = 0
multiplicity = 1
basis = 'sto-3g'

The next step is construction of `ProblemSet` instance - object containing all the information and necessary method for
our electronic structure problem.

In [ ]:
import saoovqe

problem = saoovqe.problem.ProblemSet(geometry=geometry,
                                     charge=charge,
                                     multiplicity=multiplicity,
                                     n_electrons_active=n_elec_active,
                                     n_orbitals_active=n_orbs_active,
                                     basis_name=basis)

Now we need to create a set of circuits representing orthogonal states to construct the whole circuits representing
state vectors later.

In [ ]:
initial_circuits = saoovqe.OrthogonalCircuitSet.from_problem_set(n_states=2, problem=problem)

The next necessary part is to define an ansatz - it'll be also used to construct the state vector circuits later.

In [ ]:
ansatz = saoovqe.Ansatz.from_ProblemSet(ansatz=saoovqe.AnsatzType.GUCCSD,
                                        problem=problem,
                                        repetitions=1,
                                        qubit_mapper=problem.fermionic_mapper)

And finally, now we can create an instance of our SA-OO-VQE solver. One of the main points is, orbital-optimization
can, but doesn't have to be used, or it can be used only on some molecular orbitals. We'll show all three cases here.
For no orbital-optimization it's enough to pass `None` to `orbital_optimization_settings` (it's also a default value).

Here we'll utilize `qiskit_ibm_runtime` module, where we apply for a "job" via an instance of `Session`. In this session
we create new instances of `Estimator` and `Sampler` to perform measurements and the rest of the code stays the same.

In [ ]:
from qiskit.algorithms.optimizers import SPSA
from qiskit_ibm_runtime import Estimator, Sampler, Session

optimizer=SPSA()

with Session(backend='ibmq_qasm_simulator') as session:
    print('Session is active.')

    solver_no_oo = saoovqe.SAOOVQE(estimator=Estimator(session=session),
                                   sampler=Sampler(session=session),
                                   initial_circuits=initial_circuits,
                                   ansatz=ansatz,
                                   problem=problem,
                                   orbital_optimization_settings=None)
    energies_no_oo = solver_no_oo.get_energy(optimizer)

    print('\n============== H2 Energies ==============')
    print(energies_no_oo)